# Welcome to the 210Pb age model script!

### <div style="text-align: right"> Last modified by A.A. Lehrmann 15 April 2025 </div>


### The script below will extract radioisotope data from Canberra PDFs, run the age model (from the Wellner Lab Group excel model (Appleby, 2001; Boldt et al., 2013), and plot the age model

### Important instructions before you begin:

    1. NEVER edit raw data. Do not delete Canberra PDFs. Do not remove sediment weights from original lab notebook excel sheet.

    2. Make an /CORE_AgeModelOutput/ folder to put all of your script's outputs

    3. When copying folder paths, make sure to remove quotation marks

    4. Always add the extension .csv to your output files

## First, extract radioisotope data from Canberra PDFs
Run cell (press triangle that says run) and follow instructions.

In [ ]:
from canberra_functions import extract_pdf_values, process_activity

In [ ]:
#User inputs
folder_path = input("Enter the folder path of Canberra PDFs: ")
output_csv_path = input("Enter the output CSV file path (e.g. CORE_CanberraData_DATE.csv): ")

In [ ]:
#Execute functions
extract_pdf_values(folder_path, output_csv_path)

### Make note of which samples are missing data! This will be important when we plot!

# Open the output .csv file

Check to make sure all radioisotope data translated correctly. 

# Create two new columns
- ptsrc_pb210
- ptsrc_pb210 error

# Move Point Source Lead 210 data to ptsrc_pb210 and uncertainty to ptsrc_pb210 error of associated samples

# CHECK the following

Radioisotope data should have the following headings

 ### | File    | Pb-210   | Pb-210 error    | Bi-214  | Bi-214 error   | Pb-214    |Pb-214 error |  ptsrc_pb210    | ptsrc_pb210 error  | 


# Create a new .csv file from lab notebook for the sample weight data

Sample weights should have the following headings: 

### | Core    | Top of interval (cm)   | Center point of interval    |Base of interval (cm)  | sediment weight (g)    | 




Run cell below

In [ ]:
import pandas as pd
import numpy as np
import re

# Prompt user for file paths and output file name
csv1_path = input("Enter the path to the sample weight CSV file (e.g., /path/weights.csv): ")
csv1_path = input("Enter the path to the Canberra data CSV file (e.g., /path/canberra.csv): ")
# Prompt the user for the year of core
year_of_core = int(input("Enter the year of core (e.g., 2023): "))
output_file_name = input("Enter the path for the output CSV file (e.g., CORE_AgeModel_DATE.csv): ")


In [ ]:
process_activity(csv1_path, csv1_path, output_file_name, year_of_core)

# Check the output data. Make sure data isnt *fishy*
Look at the column labeled 'Age'. Are the ages within the realm of possibility? If not, ask Asmara for help!

# Now plot it!
Run cell below 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

# Ask for the age model file to be plotted (CSV format)
age_model_file = input("Enter the full path to the age model file to plot (e.g., /path/to/age_model.csv): ")
data = pd.read_csv(age_model_file)


In [ ]:
# Get the core name for the plot title
core_name = input("Enter the core name for the title: ")

# Ask for depths to label "calendar years pre year of core"
depths_to_label_input = input("Enter the depths (comma-separated) where 'calendar years pre year of core' should be labeled (or type 'all' to label all intervals): ")

if depths_to_label_input.lower() == 'all':
    depths_to_label = data['Center point of interval'].tolist()  # Label all intervals
else:
    depths_to_label = [float(depth.strip()) for depth in depths_to_label_input.split(",")]

In [ ]:
# Ask if any intervals have undetectable radioisotope amounts
missing_data_input = input("Are there any intervals with undetectable amounts of radioisotopes? (yes/no): ").strip().lower()

if missing_data_input == 'yes':
    missing_depths_input = input("Enter the depths (comma-separated) with undetectable radioisotopes: ")
    missing_depths = [float(depth.strip()) for depth in missing_depths_input.split(",")]
else:
    missing_depths = []


In [ ]:
# Define colors for the data series and error bars
excess_pb210_color = 'black'
excess_pb210_error_color = mcolors.to_rgba(excess_pb210_color, alpha=0.3)
supported_activity_color = 'grey'
supported_activity_error_color = mcolors.to_rgba(supported_activity_color, alpha=0.3)

# Ask for the folder to save the plot PDF and create a filename
save_location = input("Enter the full path where you want to save the plot PDF (e.g., /path/to/your/directory): ")
plot_filename = f"{core_name}_Age_Model.pdf"
save_path = save_location.rstrip('/') + "/" + plot_filename

In [ ]:
plt.figure(figsize=(3, 5))
plt.errorbar(
    data['Pb-210 activity (Bq/g)'], data['Center point of interval'], 
    xerr=data['Pb-210 activity Uncertainty (Bq-g)'], fmt='-', color=excess_pb210_color, 
    label='Pb-210 activity (Bq/unit)', capsize=5, linewidth=1, 
    ecolor=excess_pb210_error_color
)
plt.xscale('log')
plt.xlim(0.01, 10)
# Highlight intervals with missing radioisotopes using brown spans
for y in missing_depths:
    plt.axhspan(y - 0.5, y + 0.5, alpha=0.5, color='brown', 
                label='Undetectable radioisotope' if y == missing_depths[0] else None)

plt.title(f"{core_name} 210 Pb Uncorrected Activity", fontsize=18)
plt.xlabel("Bq/g", fontsize=14)
plt.ylabel("Depth (cm)", fontsize=14)
plt.gca().invert_yaxis()  # Show depth from surface (invert y-axis)
plt.grid(True, which='both', linestyle='-', linewidth=0.5, color='lightgray')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=1)
plt.tight_layout()
plt.savefig(save_path, format='pdf', bbox_inches='tight')
plt.show()


In [ ]:
plt.figure(figsize=(5, 10))

# Plot Excess Pb-210 (Bq/g) with error bars
plt.errorbar(
    data['Excess Pb-210 (Bq/g)'], data['Center point of interval'], 
    xerr=data['Pb-210 activity Uncertainty (Bq-g)'], fmt='-', color=excess_pb210_color, 
    label='Excess Pb-210', capsize=5, linewidth=1, 
    ecolor=excess_pb210_error_color
)

# Plot Averaged supported activity with error bars
plt.errorbar(
    data['Averaged supported activity of Bi-214 and Pb-214 (Bq/g)'], 
    data['Center point of interval'], 
    xerr=data['Background activity uncertainty (Bq/g)'], fmt='-', 
    color=supported_activity_color, label='Background Activity', 
    capsize=5, linewidth=1, ecolor=supported_activity_error_color
)
plt.xscale('log')
plt.xlim(0.01, 10)

# Highlight missing intervals with brown spans
for y in missing_depths:
    plt.axhspan(y - 0.5, y + 0.5, alpha=0.5, color='brown', 
                label='Undetectable radioisotope' if y == missing_depths[0] else None)

# Annotate the selected depths with "calendar years pre year of core"
for i, depth in enumerate(data['Center point of interval']):
    if depth in depths_to_label:
        year_value = data['calendar years pre year of core'].iloc[i]
        if not pd.isna(year_value):
            plt.text(
                data['Excess Pb-210 (Bq/g)'].iloc[i] + 0.05, depth,
                f'{int(year_value)}', fontsize=14, color='black', verticalalignment='center'
            )

plt.title(f"{core_name} Age Model", fontsize=18)
plt.xlabel("Bq/unit", fontsize=14)
plt.ylabel("Depth (cm)", fontsize=14)
plt.gca().invert_yaxis()  # Invert y-axis for depth from surface
plt.grid(True, which='both', linestyle='-', linewidth=0.5, color='lightgray')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=1)
plt.tight_layout()
plt.savefig(save_path, format='pdf', bbox_inches='tight')
plt.show()


# Well done!

#### When you've finished, go to Cell > All Output > Clear to be ready for the next user of this script.